### Lab_7_Inclass_Assignment
- Name: Paras Rupani
- Student ID: 8961758

##### Data Integration: Approximate String Matching
1- Implement FROM SCRATCH the approximate string matching functionality using Levenshtein Distance
- Levenshtein Distance between two strings a,b of length |a| and |b| is calculated by creating a matrix of dimensions |a+1|x|b+1| and then filling the matrix by the following equation:<br>
lev<sub>a,b</sub>(i,j) = <br>
a. max(i,j) IF min(i,j)=0<br>
b. minimum of:    
    <ol> <li>lev<sub>a,b</sub>(i-1,j)+1</li>
    <li>lev<sub>a,b</sub>(i,j-1)+1</li>
    <li>lev<sub>a,b</sub>(i-1,j-1)+ 
            *1 if (a<sub>i</sub> != b<sub>j</sub>), 0 otherwise*</li> </ol>

In [1]:
import pandas as pd

In [2]:
def calculate_levenshtein_distance(string_a, string_b):
    ## TODO: Write the functionality that calculates how many edit operations are required to change one string into another
    ## Example: Distance "HONDA" to "HYUNDAI" is 3, where the operations are adding "Y" after H in Honda
    ## Then replacing O with Y, then keeping N, keeping D, keeping A, adding I.
    len_word_a = len(string_a)
    len_word_b = len(string_b)

    # Creating the matrix to store the distances
    distance = [[0 for i in range(len_word_a + 1)] for i in range(len_word_b + 1)]

    # Filling the values of the 1st row and column of the matrix
    for i in range(len_word_b + 1): 
        distance[i][0] = i
        
    for j in range(len_word_a + 1): 
        distance[0][j] = j

    # Creating a directory to store the 
    new_word = {}
    
    # Applying Levenshtein Distance formula
    for i in range(1, len_word_b + 1):
        for j in range(1, len_word_a + 1):
            sub_value = 1 if string_b[i - 1] != string_a[j - 1] else 0
            option_1 = distance[i - 1][j] + 1 # Deletion
            option_2 =  distance[i][j - 1] + 1 # Insertion
            option_3 = distance[i - 1][j - 1] + sub_value # Substitution
            
            options = min(option_1, option_2, option_3)
            distance[i][j] = options
            
            # Record the edit operation in the dictionary
            if options == option_1:
                new_word[(i - 1, j)] = f"Deleted '{string_a[j-1]}'"
            elif options == option_2:
                new_word[(i, j - 1)] = f"Inserted '{string_b[i-1]}'"
            elif options == option_3 and sub_value > 0:
                new_word[(i - 1, j - 1)] = f"Substituted '{string_a[j-1]}' with '{string_b[i-1]}'"
            
    # Getting the levenshtein distance
    levenshtein_distance = distance[len_word_b][len_word_a]
    
    return levenshtein_distance, new_word

## Implementing Levenshtein Distance

In [3]:
first_word = "HONDA"
sec_word = "HYUNDAI" 
distance, iteration = calculate_levenshtein_distance(first_word, sec_word)

print(f"Levenshtein Distance between {first_word} and {sec_word} is", distance)
print()

for i,j in iteration.items():
    print(f"{i}: {j}")

Levenshtein Distance between HONDA and HYUNDAI is 3

(1, 1): Substituted 'O' with 'Y'
(1, 2): Inserted 'H'
(1, 3): Inserted 'H'
(1, 4): Inserted 'H'
(2, 2): Substituted 'N' with 'U'
(2, 3): Inserted 'Y'
(2, 4): Inserted 'Y'
(2, 1): Deleted 'H'
(3, 3): Inserted 'U'
(3, 4): Inserted 'U'
(3, 1): Deleted 'H'
(3, 2): Deleted 'O'
(4, 3): Deleted 'N'
(4, 4): Inserted 'N'
(4, 1): Deleted 'H'
(4, 2): Deleted 'O'
(5, 4): Deleted 'D'
(5, 1): Deleted 'H'
(5, 2): Deleted 'O'
(5, 3): Deleted 'N'
(6, 1): Deleted 'H'
(6, 2): Deleted 'O'
(6, 3): Deleted 'N'
(6, 4): Deleted 'D'
(6, 5): Deleted 'A'


2- Using the implemented Levenshtein's distance, merge the following dataframes based on the closest match for the "Name" attribute. (You will need to find first the matching pairs)

In [4]:
# First Dataframe
data1 = {'ID1': [1, 2, 3, 4],
         'Name1': ['John Smith', 'Alice Johnson', 'Bob Marley', 'David Doe']}
df1 = pd.DataFrame(data1)
# Second Dataframe
data2 = {'ID2': [101, 102, 103, 104,105],
         'Name2': ['Jon Smith', 'Alyce Jonson', 'Robert Marley', 'Dave D.', 'Rob Marly']}
df2 = pd.DataFrame(data2)

In [5]:
# A list which holds closest match
closest_match = []

for index_1, i in df1.iterrows():
    cur_match = None

    # Initilizing Distance to Infinty
    min_dist = float('inf') 
    for index_2, j in df2.iterrows():

        # Calculating Levenshtein Distance
        distance_1,operation_1 = calculate_levenshtein_distance(i['Name1'], j['Name2'])

        if distance < min_dist:
            min_dist = distance_1
            cur_match = index_2
    closest_match.append((index_1, cur_match))

print(closest_match)

[(0, 0), (1, 1), (2, 4), (3, 4)]


In [6]:
match_values = []

for match in closest_match:
    value_1, value_2 = match
    i = df1.loc[value_1]
    j = df2.loc[value_2]
    match_values.append([i['ID1'], i['Name1'], j['ID2'], j['Name2']])

match_data = pd.DataFrame(match_values, columns=['ID1', 'Name1', 'ID2', 'Name2'])
match_data

,ID1,Name1,ID2,Name2
0,1,John Smith,101,Jon Smith
1,2,Alice Johnson,102,Alyce Jonson
2,3,Bob Marley,105,Rob Marly
3,4,David Doe,105,Rob Marly


3- Using bookings and yachts csv:
<ol>
    <li> Inspect the bookings and yachts csv files.</li>
    <li> Before merging, make sure to keep only bookings with correctly formatted Yacht_IDs, where the Yacht_ID should be in a format of two characters, followed by 3 numbers.</li>
    <li> Combine both dataframes into one dataframe.</li>
    <li> Make sure that all columns are of correct format where it is expected that: <br>
        a. Title, First_Name, Last_Name, Street_Address, Area, Post_Code, Telephone, Yacht_ID, Yacht_Name, Yacht_Make, Yacht_Ref, Depart_Port, Depart_Country<br>
        b. Cust_ID, Built, Length(m), Cabins, Toilets, Guiests, Day_Rate are Numbers <br>
        c. Depart_Date is Date (its original format id dd/mm/yyyy)</li>
    
</ol>

***Note: Data may contain Latin Characters, You may need to use Latin Encoding, drop data where name contains any characters other than normal alphabetics***

----
### Inspecting `bookings.csv`, `yachts.csv`

In [7]:
booking_df = pd.read_csv("./bookings.csv", encoding="latin1")
yacht_df = pd.read_csv("./yachts_2.csv", encoding="latin1")

In [8]:
booking_df.head()

,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,Yacht_ID
0,Mr,Finley,Kosar,1580,27 Culmore Cross,Marden,TN21 1ZL,77009001111,12/6/2022,14,SY0182
1,Ms,Caitlin,Anderson,1322,34 Portelet Road,Southwell,KA8 9LJ,77009001122,26/07/2022,8,SY098
2,Mr,Christopher,Washington,1323,108 Chatsworth Way,New Milton,BH25 6GY,77009001133,7/5/2022,10,SY026
3,Mrs,Nicola,Leighton,1330,108 St Maurices Road,Preston,CT3 8BR,77009001152,7/5/2022,14,SY118
4,Mr,David,Ellis,1338,15 Omega Place,Hatfield,ML6 0XP,77009001154,23/06/2022,14,SY094


In [9]:
booking_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 979 entries, 0 to 978
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Title           979 non-null    object
 1   First_Name      979 non-null    object
 2   Last_Name       979 non-null    object
 3   Cust_ID         979 non-null    int64 
 4   Street_Address  979 non-null    object
 5   Area            979 non-null    object
 6   Post_Code       979 non-null    object
 7   Telephone       979 non-null    int64 
 8   Depart_Date     979 non-null    object
 9   Days            979 non-null    int64 
 10  Yacht_ID        979 non-null    object
dtypes: int64(3), object(8)
memory usage: 84.3+ KB


In [10]:
yacht_df.head()

,Yacht_Name,Yacht_Make,Yacht_Ref,Depart_Port,Depart_Country,Built,Length(m),Cabins,Toilets,Guests,Day_Rate
0,Adriatic Star,Balandro Calypso 45,SY001,Pavao Island Marina,Croatia,2013,13.85,4,2,10,656
1,Amber Sunrise,Benedetti 37 Mirage,SY002,Pavao Island Marina,Croatia,2014,11.30,3,1,8,431
2,Anne Elizabeth,Marchal Grande 410,SY003,Port Djurak,Croatia,2016,12.35,3,2,8,486
3,Aqua Calma,Benedetti 37 Mirage,SY004,Marina Naupotiri,Greece,2007,11.35,3,1,8,377
4,Arizona Challenger,Marchal Grande 382,SY005,Pavao Island Marina,Croatia,2016,11.23,3,2,8,423


In [11]:
yacht_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Yacht_Name      119 non-null    object 
 1   Yacht_Make      119 non-null    object 
 2   Yacht_Ref       119 non-null    object 
 3   Depart_Port     119 non-null    object 
 4   Depart_Country  119 non-null    object 
 5   Built           119 non-null    int64  
 6   Length(m)       119 non-null    float64
 7   Cabins          119 non-null    int64  
 8   Toilets         119 non-null    int64  
 9   Guests          119 non-null    int64  
 10  Day_Rate        119 non-null    int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 10.4+ KB


### Dropping Names with Latin Alphabets (Other than English)

In [12]:
# Creating a pattern to match only normal alphabets
normal_alphabets = r'^[A-Za-z\s]+$'

# Filtering and removing all the unmatched data from the datasets.
booking_df = booking_df[booking_df['First_Name'].str.match(normal_alphabets)] 
booking_df = booking_df[booking_df['Last_Name'].str.match(normal_alphabets)] 

booking_df.head()

,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,Yacht_ID
0,Mr,Finley,Kosar,1580,27 Culmore Cross,Marden,TN21 1ZL,77009001111,12/6/2022,14,SY0182
1,Ms,Caitlin,Anderson,1322,34 Portelet Road,Southwell,KA8 9LJ,77009001122,26/07/2022,8,SY098
2,Mr,Christopher,Washington,1323,108 Chatsworth Way,New Milton,BH25 6GY,77009001133,7/5/2022,10,SY026
3,Mrs,Nicola,Leighton,1330,108 St Maurices Road,Preston,CT3 8BR,77009001152,7/5/2022,14,SY118
4,Mr,David,Ellis,1338,15 Omega Place,Hatfield,ML6 0XP,77009001154,23/06/2022,14,SY094


In [13]:
booking_df['Yacht_ID']

0      SY0182
1       SY098
2       SY026
3       SY118
4       SY094
        ...  
974     SY094
975     SY048
976     SY114
977     SY009
978     SY108
Name: Yacht_ID, Length: 962, dtype: object

----
### Formatting `Yacht_ID` to be two characters, followed by 3 numbers.

In [14]:
# Creating a pattern for the Yacht_ID
pattern = r'^[A-Za-z]{2}\d{3}$'

# Filtering and removing all the unmatched data from the datasets.
booking_df_df = booking_df[booking_df['Yacht_ID'].str.match(pattern, na=False)]

booking_df

,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,Yacht_ID
0,Mr,Finley,Kosar,1580,27 Culmore Cross,Marden,TN21 1ZL,77009001111,12/6/2022,14,SY0182
1,Ms,Caitlin,Anderson,1322,34 Portelet Road,Southwell,KA8 9LJ,77009001122,26/07/2022,8,SY098
2,Mr,Christopher,Washington,1323,108 Chatsworth Way,New Milton,BH25 6GY,77009001133,7/5/2022,10,SY026
3,Mrs,Nicola,Leighton,1330,108 St Maurices Road,Preston,CT3 8BR,77009001152,7/5/2022,14,SY118
4,Mr,David,Ellis,1338,15 Omega Place,Hatfield,ML6 0XP,77009001154,23/06/2022,14,SY094
...,...,...,...,...,...,...,...,...,...,...,...
974,Mr,Lewis,Lawrence,4301,92 High Street,Knaresborough,SY30 0SD,77009009974,7/9/2022,10,SY094
975,Mr,Jakob,Enger,4303,129 Grafton Road,Spalding,PE11 5TN,77009009981,29/05/2022,14,SY048
976,Mr,Benjamin,Goossens,4304,39 Globe Road,Walden,DT23 6HD,77009009987,22/07/2022,14,SY114
977,Mr,Logan,Richardson,4308,127 Whiterock Road,Tattershall,AB41 5FB,77009009988,4/6/2022,28,SY009


----
### Combining DataFrames

In [15]:
# Combine both DataFrames using pd.merge
merged_df = pd.merge(booking_df, yacht_df, left_on='Yacht_ID', right_on='Yacht_Ref')
merged_df

,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,...,Yacht_Make,Yacht_Ref,Depart_Port,Depart_Country,Built,Length(m),Cabins,Toilets,Guests,Day_Rate
0,Ms,Caitlin,Anderson,1322,34 Portelet Road,Southwell,KA8 9LJ,77009001122,26/07/2022,8,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
1,Mrs,Lauren,George,1358,85 Havant Arcade,Holywell,BN17 8EH,77009001171,21/08/2022,14,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
2,Mr,Evan,Lewis,2719,106 East Dombey Square,Richmond,CA5 3GN,77009005417,7/5/2022,28,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
3,Mr,Henry,Rhodes,3074,125 Chalfont Avenue,Ely,CT4 7ZT,77009006374,7/6/2022,16,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
4,Mr,Lucas,Rose,3131,50 Saffron Hill,Petersfield,SA12 4NW,77009006539,5/8/2022,16,...,Sevier Luna 42,SY098,Marina Vitus,Croatia,2017,12.8,6,4,12,1057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,Mr,Zachary,Hammond,3045,98 Railway Cottages,Midhurst,PL14 7SP,77009006273,11/8/2022,15,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
955,Mr,Carl,Hind,3239,15 Cavell Street,Cleeve,SA34 0WZ,77009006944,28/06/2022,28,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
956,Mrs,Magdalena,Schuster,3294,101 Brooklands Park,Shaftesbury,EH6 7BH,77009007222,21/05/2022,15,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566
957,Mr,Shaun,Peretti,3358,146 Atwood Road,Chigwell,LL22 8BT,77009007412,6/6/2022,21,...,Benedetti 46 Alto,SY025,Pavao Island Marina,Croatia,2007,14.4,4,2,9,566


Looking at the various data types of all the columns

In [16]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 959 entries, 0 to 958
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Title           959 non-null    object 
 1   First_Name      959 non-null    object 
 2   Last_Name       959 non-null    object 
 3   Cust_ID         959 non-null    int64  
 4   Street_Address  959 non-null    object 
 5   Area            959 non-null    object 
 6   Post_Code       959 non-null    object 
 7   Telephone       959 non-null    int64  
 8   Depart_Date     959 non-null    object 
 9   Days            959 non-null    int64  
 10  Yacht_ID        959 non-null    object 
 11  Yacht_Name      959 non-null    object 
 12  Yacht_Make      959 non-null    object 
 13  Yacht_Ref       959 non-null    object 
 14  Depart_Port     959 non-null    object 
 15  Depart_Country  959 non-null    object 
 16  Built           959 non-null    int64  
 17  Length(m)       959 non-null    flo

----
### Formatting the Columns of the Dataset

- ####   a. Title, First_Name, Last_Name, Street_Address, Area, Post_Code, Telephone, Yacht_ID, Yacht_Name, Yacht_Make, Yacht_Ref, Depart_Port, Depart_Country

In [17]:
merged_df.select_dtypes('object').columns

Index(['Title', 'First_Name', 'Last_Name', 'Street_Address', 'Area',
       'Post_Code', 'Depart_Date', 'Yacht_ID', 'Yacht_Name', 'Yacht_Make',
       'Yacht_Ref', 'Depart_Port', 'Depart_Country'],
      dtype='object')

* ##### `Telephone` should be in the object format.
* ##### `Depart_Date` should not be in object format.


In [18]:
merged_df['Telephone'] = merged_df['Telephone'].astype(str)
merged_df['Depart_Date'] = pd.to_datetime(merged_df['Depart_Date'], format="%d/%M/%Y")
merged_df.select_dtypes('object').columns

Index(['Title', 'First_Name', 'Last_Name', 'Street_Address', 'Area',
       'Post_Code', 'Telephone', 'Yacht_ID', 'Yacht_Name', 'Yacht_Make',
       'Yacht_Ref', 'Depart_Port', 'Depart_Country'],
      dtype='object')

----
### Numerical Columns in the Dataset
* #### Cust_ID, Built, Length(m), Cabins, Toilets, Guiests, Day_Rate are Numbers

In [19]:
merged_df.select_dtypes(['int', 'float']).columns

Index(['Cust_ID', 'Days', 'Built', 'Length(m)', 'Cabins', 'Toilets', 'Guests',
       'Day_Rate'],
      dtype='object')

* ##### `Days` is the only column which is included as numeric, converting it into non-numeric. 

In [20]:
merged_df['Days'] = merged_df['Days'].astype(str)

In [21]:
merged_df.select_dtypes(['int', 'float']).columns

Index(['Cust_ID', 'Built', 'Length(m)', 'Cabins', 'Toilets', 'Guests',
       'Day_Rate'],
      dtype='object')

----
### `Depart_Date` is `date` format.

In [22]:
merged_df['Depart_Date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 959 entries, 0 to 958
Series name: Depart_Date
Non-Null Count  Dtype         
--------------  -----         
959 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 7.6 KB


* #### Depart_Date (dd/mm/yyyy)

In [23]:
merged_df['Depart_Date'] = pd.to_datetime(merged_df['Depart_Date'], format="%d/%M/%Y")
merged_df['Depart_Date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 959 entries, 0 to 958
Series name: Depart_Date
Non-Null Count  Dtype         
--------------  -----         
959 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 7.6 KB


-----